<center>Национальный исследовательский университет ИТМО<br/>Факультет информационных технологий и программирования<br/>Прикладная математика и информатика</center>

## <center>Математическая статистика</center>
### <center>Отчёт по лабораторной работе №4</center>

<div style="text-align: right"><b>Работу выполнили:</b><br/>Белоцерковченко Андрей M3337<br/>Смирнов Андрей M3337<br/</div>

<center>Санкт-Петербург<br/>2023</center>

# Вариант 2

In [77]:
import numpy as np
from scipy.special import comb
from scipy.stats import norm
from scipy.stats import rankdata
from scipy.stats import chi2
import pandas as pd
import math
df = pd.read_csv('exams_dataset.csv')

## Задача 2
В задаче мы хотим проверить, что выборки результатов по математике и письменной части имеют одинаковое распределение. Более формально: $$H_0: F_X = F_Y$$ $$H_1: F_X \neq F_Y $$

### Критерий знаков
Пусть результаты по математике будут равны $X_1, X_2, X_3 ... X_n$, а по письменной части $Y_1, Y_2, Y_3 ... Y_n$. Возьмём последовательность $Z$, такую что $sign(X_1 - Y_1), sign(X_2 - Y_2), sign(X_3 - Y_3) ... sign(X_n - Y_n)$. Выкинем из рассмотрения все нули (случаи, когда $X_i = Y_i$). Мы хотим проверить, правда ли, что $P(Z > 0) = 0.5$.
Сформулируем гипотезы: $$H_0: p = \cfrac{1}{2}$$ $$H_1: p \neq \cfrac{1}{2}$$
Пусть $w$ - количество $Z_i = 1$. Если $H_0$ выполняется, то $w \sim B(n, 0.5)$ А дальше просто проверяем, что $b = \cfrac{\sum\limits_{i = 0}^w C_n^i}{2^n}$ попадает в критическую область $(\cfrac{\alpha}{2}, 1 - \cfrac{\alpha}{2})$, где $\alpha$ - уровень значимости равный $0.01$

In [72]:
sign = lambda x: math.copysign(1, x)
test = (df['math score'] - df['writing score']).apply(sign)
w = test.gt(0).sum()
cSum = 0
for i in range(0, w + 1):
    cSum += comb(1000, i, exact=True)

b = cSum / (2 ** 1000)
print("b:", b)
print("Check if b in [alpha / 2, 1 - alpha / 2]:", (0.05 / 2) < b < 1 - (0.05 / 2))

b: 0.0024308681959370596
Check if b in [alpha / 2, 1 - alpha / 2]: False


## Критерий Ван дер Вардена



In [81]:
def count_mean_rank(f, s, d):
    n = len(f)
    m = len(s)
    k = n + m + 1
    R = 0
    for x in f:
        R += norm.ppf(d[x] / k)
        
    return R / n

def count_varience(f, s):
    n = len(f)
    m = len(s)
    
    R = 0
    for x in f:
        R += x
    for y in s:
        R += y
        
    return R / (n + m - 1)

a = df['math score']
b = df['writing score']
u = np.concatenate((a, b))
number_to_rank = dict(zip(u, rankdata(u)))
stat = ((count_mean_rank(a, b, number_to_rank) ** 2) * len(a) + 
        (count_mean_rank(b, a, number_to_rank) ** 2) * len(b)) / count_varience(a, b)
print("Van der Waerden stat:", stat)
print("Check if stat in [chi2_alpha / 2, 1 - chi2_alpha / 2]", chi2.ppf(0.01, 1) < stat < chi2.ppf(0.99, 1))
print(chi2.ppf(0.05, 1))
print(chi2.ppf(0.95, 1))

Van der Waerden stat: 0.10906549996717955
Check if stat in [chi2_alpha / 2, 1 - chi2_alpha / 2] True
0.003932140000019522
3.841458820694124


Таким образом, оба критерия показали неоднородность результатов эказмена по математике и письменной части.  